In [ ]:
import pandas as pd

In [5]:
# import the data
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
full_aggregated_data = pd.read_csv("counts_per_intersections.csv")

In [8]:
median_columns = [x for x in full_aggregated_data.columns if x.endswith("_median")]
median_columns

['n_cameras_median',
 'attached_street_median',
 'attached_building_median',
 'attached_unknown_median',
 'type_dome_median',
 'type_bullet_median',
 'type_unknown_median']

In [ ]:
final_long = full_aggregated_data.reset_index().melt(id_vars = ['PanoramaId', 'BoroName'],
                                 value_vars = median_columns,
                                 var_name = "subset",
                                 value_name = "count"
                                )
final_long["subset"] = final_long["subset"].str.replace("_median", "")

In [ ]:
by_borough = final_long[final_long["subset"].str.startswith("attached_")].groupby(["BoroName", "subset"]).sum()
by_borough["Percentage"] = by_borough.groupby(level=0).apply(lambda x: 100*x / x.sum())

In [14]:
public_per_borough = by_borough.xs("attached_street", level="subset")["count"]
public_per_borough

BoroName
Bronx             456
Brooklyn          871
Manhattan         724
Queens           1088
Staten Island     178
Name: count, dtype: int64

In [15]:
public_per_borough.sum()

3317

In [ ]:
total_per_borough = by_borough.groupby(level=0).sum()

In [17]:
total_per_borough.sum()

count         25532.0
Percentage      500.0
dtype: float64

In [18]:
intersections_per_borough = final_long.groupby("BoroName")["PanoramaId"].nunique()
intersections_per_borough

BoroName
Bronx             6257
Brooklyn         10384
Manhattan         3934
Queens           16344
Staten Island     6474
Name: PanoramaId, dtype: int64

In [19]:
cameras_per_intersection = total_per_borough["count"] / intersections_per_borough
cameras_per_intersection

BoroName
Bronx            0.597251
Brooklyn         0.889060
Manhattan        1.003559
Queens           0.463779
Staten Island    0.159870
dtype: float64

In [20]:
cameras_per_intersection.sum()

3.1135188970648806

In [21]:
public_per_intersection = public_per_borough / intersections_per_borough
public_per_intersection

BoroName
Bronx            0.072878
Brooklyn         0.083879
Manhattan        0.184037
Queens           0.066569
Staten Island    0.027495
dtype: float64

In [22]:
public_per_intersection.sum()

0.43485739004243457

In [24]:
numofcamera = pd.DataFrame(total_per_borough)

In [26]:
densityofcamera = pd.DataFrame(cameras_per_intersection)

In [28]:
numofcamera['Density']=densityofcamera[0]

In [30]:
from google.colab import files
numofcamera.to_csv('numner_of_camera_by_boro.csv')
files.download('numner_of_camera_by_boro.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>